In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def poblacion_inactiva_sin_ingresos(df, tipo='Aglomerado', base='Individual', confidence_level=0.95, pool=False):

    # En este caso dividir el indicador segun algun percentil de ingresos. Por ejemplo de 90 a 60. Ver literatura al respecto 

    """
    INPUTS
    df: DataFrame. Tabla input EPH
    tipo: string. Tipo de encuesta de la EPH, Aglomerado o Urbano. Default Aglomerado
    base: string. Tipo de base de la encuesta de la EPH, Individual u Hogar. Default Individual

    OUTPUTS
    ratio: DataFrame. Tabla con Ratios en tasa M/V de población inactiva que no estudia y no tiene ingresos propios, desagregado por Aglomerado o Provincia
    error: DataFrame. Tabla con los errores asociados a los Ratios

    """

    if tipo == 'Aglomerado':
        var = 'AGLOMERADO'
    elif tipo == 'Urbano':
        var = 'PROVINCIA'

    df_temp = df.query('CH06 >= 14')
    
    if pool:
        pondera = 'PONDII_new'
        p47t = 'P47T_new'
    else:
        pondera = 'PONDII'
        p47t = 'P47T'

    df_inactivos = df_temp[(df_temp[p47t]!=-9) & (df_temp['ESTADO']==3) & (df_temp['CAT_INAC']!=3)][[var, pondera, p47t, 'CH04']]
    numerador = df_inactivos[(df_inactivos[p47t]==0)].groupby(['CH04', var])[pondera].sum().unstack(level=0)
    denominador = df_inactivos.groupby(['CH04', var])[pondera].sum().unstack(level=0)

    tasa = numerador.div(denominador, fill_value=np.nan)
    tasa.columns = ['Varon', 'Mujer']

    size = df_inactivos[(df_inactivos[p47t]==0)].groupby(['CH04', var]).size().to_frame().unstack(level=0)
    size.columns = ['N_v', 'N_m']

    p1 = tasa['Varon']
    p2 = tasa['Mujer']
    n1 = size['N_v']
    n2 = size['N_m']
    ratio = (p1 / p2).to_frame()
    ratio.rename(columns={0: 'Dependencia'}, inplace=True)
    
    # Calculate the standard error of the proportion ratio
    standard_error = np.sqrt(((1 / n1) * (p1 * (1 - p1))) + ((1 / n2) * (p2 * (1 - p2))))

    z = stats.norm.ppf((1 + confidence_level) / 2)
    margin_of_error = z * standard_error
    lower_bound = p1/p2 - margin_of_error
    upper_bound = p1/p2 + margin_of_error

    relative_standard_error = margin_of_error/(p1/p2)
    error = pd.concat([lower_bound, upper_bound, margin_of_error, relative_standard_error], axis=1)
    error.columns = ['LI', 'LS', 'ME', 'ER']

    return tasa*100, size, ratio*100, error*100


In [ ]:
def poblacion_inactiva_con_ingresos(df, tipo='Aglomerado', base='Individual', confidence_level=0.95, pool=False):

    # En este caso dividir el indicador segun algun percentil de ingresos. Por ejemplo de 90 a 60. Ver literatura al respecto 

    """
    INPUTS
    df: DataFrame. Tabla input EPH
    tipo: string. Tipo de encuesta de la EPH, Aglomerado o Urbano. Default Aglomerado
    base: string. Tipo de base de la encuesta de la EPH, Individual u Hogar. Default Individual

    OUTPUTS
    ratio: DataFrame. Tabla con Ratios en tasa M/V de población inactiva que no estudia y no tiene ingresos propios, desagregado por Aglomerado o Provincia
    error: DataFrame. Tabla con los errores asociados a los Ratios

    """

    if tipo == 'Aglomerado':
        var = 'AGLOMERADO'
    elif tipo == 'Urbano':
        var = 'PROVINCIA'

    df_temp = df.query('CH06 >= 14')
    
    if pool:
        pondera = 'PONDII_new'
        p47t = 'P47T_new'
    else:
        pondera = 'PONDII'
        p47t = 'P47T'

    df_inactivos = df_temp[(df_temp[p47t]!=-9) & (df_temp['ESTADO']==3) & (df_temp['CAT_INAC']!=3)][[var, pondera, p47t, 'CH04']]
    numerador = df_inactivos[(df_inactivos[p47t]>0)].groupby(['CH04', var])[pondera].sum().unstack(level=0)
    denominador = df_inactivos.groupby(['CH04', var])[pondera].sum().unstack(level=0)

    tasa = numerador.div(denominador, fill_value=np.nan)
    tasa.columns = ['Varon', 'Mujer']

    size = df_inactivos[(df_inactivos[p47t]>0)].groupby(['CH04', var]).size().to_frame().unstack(level=0)
    size.columns = ['N_v', 'N_m']

    p1 = tasa['Mujer']
    p2 = tasa['Varon']
    n1 = size['N_m']
    n2 = size['N_v']
    ratio = (p1 / p2).to_frame()
    ratio.rename(columns={0: 'Dependencia'}, inplace=True)
    
    # Calculate the standard error of the proportion ratio
    standard_error = np.sqrt(((1 / n1) * (p1 * (1 - p1))) + ((1 / n2) * (p2 * (1 - p2))))

    z = stats.norm.ppf((1 + confidence_level) / 2)
    margin_of_error = z * standard_error
    lower_bound = p1/p2 - margin_of_error
    upper_bound = p1/p2 + margin_of_error

    relative_standard_error = margin_of_error/(p1/p2)
    error = pd.concat([lower_bound, upper_bound, margin_of_error, relative_standard_error], axis=1)
    error.columns = ['LI', 'LS', 'ME', 'ER']

    return tasa*100, size, ratio*100, error*100


In [ ]:
def poblacion_inactiva_sin_ingresos_sin_65(df, tipo='Aglomerado', base='Individual', confidence_level=0.95, pool=False):

    # En este caso dividir el indicador segun algun percentil de ingresos. Por ejemplo de 90 a 60. Ver literatura al respecto 

    """
    INPUTS
    df: DataFrame. Tabla input EPH
    tipo: string. Tipo de encuesta de la EPH, Aglomerado o Urbano. Default Aglomerado
    base: string. Tipo de base de la encuesta de la EPH, Individual u Hogar. Default Individual

    OUTPUTS
    ratio: DataFrame. Tabla con Ratios en tasa M/V de población inactiva que no estudia y no tiene ingresos propios, desagregado por Aglomerado o Provincia
    error: DataFrame. Tabla con los errores asociados a los Ratios

    """

    if tipo == 'Aglomerado':
        var = 'AGLOMERADO'
    elif tipo == 'Urbano':
        var = 'PROVINCIA'

    df_temp = df.query('CH06 >= 14 & CH06 < 65')
    
    if pool:
        pondera = 'PONDII_new'
        p47t = 'P47T_new'
    else:
        pondera = 'PONDII'
        p47t = 'P47T'

    df_inactivos = df_temp[(df_temp[p47t]!=-9) & (df_temp['ESTADO']==3) & (df_temp['CAT_INAC']!=3)][[var, pondera, p47t, 'CH04']]
    numerador = df_inactivos[(df_inactivos[p47t]==0)].groupby(['CH04', var])[pondera].sum().unstack(level=0)
    denominador = df_inactivos.groupby(['CH04', var])[pondera].sum().unstack(level=0)

    tasa = numerador.div(denominador, fill_value=np.nan)
    tasa.columns = ['Varon', 'Mujer']

    size = df_inactivos[(df_inactivos[p47t]==0)].groupby(['CH04', var]).size().to_frame().unstack(level=0)
    size.columns = ['N_v', 'N_m']

    p1 = tasa['Varon']
    p2 = tasa['Mujer']
    n1 = size['N_v']
    n2 = size['N_m']
    ratio = (p1 / p2).to_frame()
    ratio.rename(columns={0: 'Dependencia'}, inplace=True)
    
    # Calculate the standard error of the proportion ratio
    standard_error = np.sqrt(((1 / n1) * (p1 * (1 - p1))) + ((1 / n2) * (p2 * (1 - p2))))

    z = stats.norm.ppf((1 + confidence_level) / 2)
    margin_of_error = z * standard_error
    lower_bound = p1/p2 - margin_of_error
    upper_bound = p1/p2 + margin_of_error

    relative_standard_error = margin_of_error/(p1/p2)
    error = pd.concat([lower_bound, upper_bound, margin_of_error, relative_standard_error], axis=1)
    error.columns = ['LI', 'LS', 'ME', 'ER']

    return tasa*100, size, ratio*100, error*100


In [ ]:
def poblacion_inactiva_con_ingresos_sin_65(df, tipo='Aglomerado', base='Individual', confidence_level=0.95, pool=False):

    # En este caso dividir el indicador segun algun percentil de ingresos. Por ejemplo de 90 a 60. Ver literatura al respecto 

    """
    INPUTS
    df: DataFrame. Tabla input EPH
    tipo: string. Tipo de encuesta de la EPH, Aglomerado o Urbano. Default Aglomerado
    base: string. Tipo de base de la encuesta de la EPH, Individual u Hogar. Default Individual

    OUTPUTS
    ratio: DataFrame. Tabla con Ratios en tasa M/V de población inactiva que no estudia y no tiene ingresos propios, desagregado por Aglomerado o Provincia
    error: DataFrame. Tabla con los errores asociados a los Ratios

    """

    if tipo == 'Aglomerado':
        var = 'AGLOMERADO'
    elif tipo == 'Urbano':
        var = 'PROVINCIA'

    df_temp = df.query('CH06 >= 14 & CH06 < 65')

    if pool:
        pondera = 'PONDII_new'
        p47t = 'P47T_new'
    else:
        pondera = 'PONDII'
        p47t = 'P47T'

    df_inactivos = df_temp[(df_temp[p47t]!=-9) & (df_temp['ESTADO']==3) & (df_temp['CAT_INAC']!=3)][[var, pondera, p47t, 'CH04']]
    numerador = df_inactivos[(df_inactivos[p47t]>0)].groupby(['CH04', var])[pondera].sum().unstack(level=0)
    denominador = df_inactivos.groupby(['CH04', var])[pondera].sum().unstack(level=0)

    tasa = numerador.div(denominador, fill_value=np.nan)
    tasa.columns = ['Varon', 'Mujer']

    size = df_inactivos[(df_inactivos[p47t]>0)].groupby(['CH04', var]).size().to_frame().unstack(level=0)
    size.columns = ['N_v', 'N_m']

    p1 = tasa['Mujer']
    p2 = tasa['Varon']
    n1 = size['N_m']
    n2 = size['N_v']
    ratio = (p1 / p2).to_frame()
    ratio.rename(columns={0: 'Dependencia'}, inplace=True)
    
    # Calculate the standard error of the proportion ratio
    standard_error = np.sqrt(((1 / n1) * (p1 * (1 - p1))) + ((1 / n2) * (p2 * (1 - p2))))

    z = stats.norm.ppf((1 + confidence_level) / 2)
    margin_of_error = z * standard_error
    lower_bound = p1/p2 - margin_of_error
    upper_bound = p1/p2 + margin_of_error

    relative_standard_error = margin_of_error/(p1/p2)
    error = pd.concat([lower_bound, upper_bound, margin_of_error, relative_standard_error], axis=1)
    error.columns = ['LI', 'LS', 'ME', 'ER']

    return tasa*100, size, ratio*100, error*100


In [ ]:
def tabla_ingresos_inactivos_no_estudiantes(df, pool=False):
    
    if pool:
        pondera = 'PONDII_new'
    else:
        pondera = 'PONDII'    
    
    df_temp = df[df['CH06'] >= 14].copy()
    bins = [14, 30, 46, 66, df_temp['CH06'].max()]
    labels = ['14-29', '30-45', '46-65', '65+']
    df_temp['Rango edad'] = pd.cut(df_temp['CH06'], bins=bins, labels=labels, right=False)

    pondera = 'PONDERA_new'
    p47t = 'P47T_new'

    df_inactivos_con_ingreso = df_temp[(df_temp['ESTADO']==3) & (df_temp[p47t]==0) & (df_temp['CAT_INAC']!=3)].groupby(['CH04','Rango edad'])[pondera].sum().unstack(level=0)
    df_inactivos_sin_ingreso = df_temp[(df_temp['ESTADO']==3) & (df_temp[p47t]>0) & (df_temp['CAT_INAC']!=3)].groupby(['CH04','Rango edad'])[pondera].sum().unstack(level=0)
    df_inactivos_no_declara_ingreso = df_temp[(df_temp['ESTADO']==3) & (df_temp[p47t]==-9) & (df_temp['CAT_INAC']!=3)].groupby(['CH04','Rango edad'])[pondera].sum().unstack(level=0)

    n_inactivos_con_ingreso = df_temp[(df_temp['ESTADO']==3) & (df_temp[p47t]==0) & (df_temp['CAT_INAC']!=3)].groupby(['CH04','Rango edad'])[pondera].size().unstack(level=0)
    n_inactivos_sin_ingreso = df_temp[(df_temp['ESTADO']==3) & (df_temp[p47t]>0) & (df_temp['CAT_INAC']!=3)].groupby(['CH04','Rango edad'])[pondera].size().unstack(level=0)
    n_inactivos_no_declara_ingreso = df_temp[(df_temp['ESTADO']==3) & (df_temp[p47t]==-9) & (df_temp['CAT_INAC']!=3)].groupby(['CH04','Rango edad'])[pondera].size().unstack(level=0)
    
    size_colums_names = pd.MultiIndex.from_product([['Ingreso > 0', 'Ingreso = 0', 'Ingreso = -9'], ['N_v', 'N_m']])
    size_colums = pd.concat([n_inactivos_con_ingreso, n_inactivos_sin_ingreso, n_inactivos_no_declara_ingreso], axis=1)
    size_colums.loc['Total Col'] = size_colums.sum(numeric_only=True, axis=0)

    df_inactivos_con_ingreso.columns = pd.MultiIndex.from_tuples([('Ingreso > 0', 'Varon'), ('Ingreso > 0', 'Mujer')])
    df_inactivos_sin_ingreso.columns = pd.MultiIndex.from_tuples([('Ingreso = 0', 'Varon'), ('Ingreso = 0', 'Mujer')])
    df_inactivos_no_declara_ingreso.columns = pd.MultiIndex.from_tuples([('Ingreso = -9', 'Varon'), ('Ingreso = -9', 'Mujer')])

    df_inactivos = pd.concat([df_inactivos_con_ingreso, df_inactivos_sin_ingreso, df_inactivos_no_declara_ingreso], axis=1)
    df_inactivos.loc['Total Col'] = df_inactivos.sum(numeric_only=True, axis=0)

    df_inactivos_porcentaje = df_inactivos.iloc[:-1,:] / df_inactivos.iloc[-1,:] * 100
    df_inactivos_porcentaje.loc['Total Col'] = df_inactivos_porcentaje.sum(numeric_only=True, axis=0)
    tabla_pob_inactiva_porcentaje = df_inactivos_porcentaje.style.format('{:,.2f}')

    df_inactivos[size_colums_names] = size_colums.values

    df_inactivos = df_inactivos.sort_index(axis=1, ascending=False)
    tabla_pob_inactiva = df_inactivos.style.format('{:,.0f}')

    ratio_pob_inactiva = pd.DataFrame()
    ratio_pob_inactiva.loc[:,'Ingreso > 0'] = df_inactivos_porcentaje.iloc[:,1]/df_inactivos_porcentaje.iloc[:,0] * 100
    ratio_pob_inactiva.loc[:,'Ingreso = 0'] = df_inactivos_porcentaje.iloc[:,3]/df_inactivos_porcentaje.iloc[:,2] * 100
    ratio_pob_inactiva.loc[:,'Ingreso = -9'] = df_inactivos_porcentaje.iloc[:,5]/df_inactivos_porcentaje.iloc[:,4] * 100
    ratio_pob_inactiva.drop(ratio_pob_inactiva.tail(1).index, inplace=True)

    return tabla_pob_inactiva, tabla_pob_inactiva_porcentaje, ratio_pob_inactiva
